<a href="https://colab.research.google.com/github/esheshka/team_one_project/blob/Andrey/%D0%A7%D1%82%D0%BE_%D1%82%D0%BE_%D0%B7%D0%B0%D1%87%D0%B5%D0%BC_%D1%82%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [1]:
from google.colab import drive; drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 26jun23                  # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 26jun23
100% 35.7M/35.7M [00:00<00:00, 154MB/s]
Using competition: 26jun23
  teamId  teamName                   submissionDate       score    
--------  -------------------------  -------------------  -------  
10612701  Никита Угрюмов             2023-06-27 23:19:51  0.87888  
10612706  Michael Kondrashov         2023-06-27 21:06:25  0.87746  
10614699  Dmitriy Geyzha             2023-06-27 22:15:25  0.86653  
10612695  Ne1tr0N                    2023-06-27 15:03:47  0.86489  
10614521  Konstantin Zeigman         2023-06-27 22:17:38  0.85948  
10612715  Stepan Vysotsky            2023-06-28 05:41:07  0.84667  
10613713  Mikhail Vovk               2023-06-27 20:06:47  0.84485  
10612680  esheshka                   2023-06-27 20:18:28  0.68212  
10605413  💳baseline,LR:$Stat+MCCEnc  2023-06-25 18:15:10  0.53220  


In [3]:
%%time
%%capture
%reset -f
!pip install -U sentence-transformers tfds-nightly tensorflow tensorflow_addons plotly > log
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"  # allows multiple prints from a cell
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os, plotly.express as px, xgboost as xgb, lightgbm, re, tensorflow as tf, tensorflow.keras as keras
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GroupKFold

from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RFC

from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier   # SKLearn's MLP is optimised for CPU (and doesn't use GPU)
from sentence_transformers import SentenceTransformer as SBERT
from keras.layers import Flatten, Dense

os.environ['TF_DETERMINISTIC_OPS'] = '1'; os.environ['TF_CUDNN_DETERMINISTIC'] = '1'   # helps to seed TF's RNG on GPUs
np.set_printoptions(linewidth=10000, precision=2, edgeitems=20, suppress=True)
pd.set_option('display.max_colwidth', 100, 'display.max_columns', 10, 'display.width', 1000, 'display.max_rows', 8)

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60*2): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

CPU times: user 8.31 s, sys: 1.03 s, total: 9.34 s
Wall time: 1min 20s


In [4]:
#@title MCC
%time mcc = pd.read_csv('_mcc.csv', index_col='mcc'); mcc  # load merchant category codes (MCC)

CPU times: user 4.18 ms, sys: 2.07 ms, total: 6.26 ms
Wall time: 22.1 ms


,desc
mcc,
0,Ветеринарные услуги
1,"Генеральные подрядчики по вентиляции, теплоснабжению, и водопроводу"
2,Подрядчики по электричеству
3,"Подрядчики, специализированная торговля — нигде более не классифицированные"
...,...
180,Штрафы
181,Налоговые платежи
182,"Правительственные услуги, нигде ранее не классифицируемые"
183,Почтовые услуги — только правительство


In [5]:
#@title TTC
%time ttc = pd.read_csv('_ttc.csv', index_col='ttc'); ttc  # load transaction type codes (TTC)

CPU times: user 3.92 ms, sys: 0 ns, total: 3.92 ms
Wall time: 3.89 ms


,desc
ttc,
0,Покупка.ТУ СБ РФ
1,Покупка. POS ТУ СБ РФ
2,Оплата услуги. Банкоматы СБ РФ
3,Покупка. ТУ Россия
...,...
151,Плата за приост. действия карты
152,Плата за обработку счета банковской карты клиента при осущ. платежной системой экстренной выдачи...
153,Плата за предоставление по запросу клиента документов по операции с картой
154,XXX


In [6]:
#@title Gender
%time gender = pd.read_csv('gender.csv', index_col='cid'); gender.T  # load gender labels. you need to predict the missing labels
trainY, testY = gender.query('gender==gender'), gender.query('gender!=gender')  # split into Train and test (=Validation) targets

CPU times: user 7.17 ms, sys: 950 µs, total: 8.12 ms
Wall time: 9.03 ms


cid,0,1,2,3,4,...,8395,8396,8397,8398,8399
gender,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0


In [192]:
#@title TRX
%time dfTrx = pd.read_csv('trx.csv').sort_values(['cid','dt','tm']); dfTrx  # load bank card's transactions. Ensure ordered CID

CPU times: user 7.38 s, sys: 1.85 s, total: 9.23 s
Wall time: 10.6 s


,cid,dt,tm,amt,mcc,ttc,tid
0,0,1,00:00:00,-1305,56,4,NaN
1,0,1,00:00:00,-4821,56,4,NaN
2,0,2,22:48:17,-56156,22,37,NaN
3,0,3,00:00:00,-3708,56,4,NaN
4,0,3,00:00:00,-30912,65,4,NaN
...,...,...,...,...,...,...,...
3749573,8399,450,00:00:00,-21877,56,4,25021545
3749574,8399,450,00:00:00,-43831,56,4,PS026577
3749575,8399,456,17:21:07,89833,123,128,888917
3749576,8399,456,18:59:26,247062,124,108,738051


In [33]:
df_ = gender.drop('gender', axis=1); df_.T  # a list of ordered client IDs (as index)

cid,0,1,2,3,4,5,6,7,8,9,...,8390,8391,8392,8393,8394,8395,8396,8397,8398,8399


In [34]:
pd.set_option('display.max_colwidth', 100, 'display.max_columns', 20, 'display.width', 1000, 'display.max_rows', 10)
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

⏳ started. You have 120 sec. Good luck!


# **Тесты**

# **My Code**

In [141]:
dfTrx

,cid,dt,tm,amt,mcc,ttc,tid
0,0,1,0,-1305,56,4,NaN
1,0,1,0,-4821,56,4,NaN
2,0,2,22,-56156,22,37,NaN
3,0,3,0,-3708,56,4,NaN
4,0,3,0,-30912,65,4,NaN
...,...,...,...,...,...,...,...
3749573,8399,450,0,-21877,56,4,25021545
3749574,8399,450,0,-43831,56,4,PS026577
3749575,8399,456,17,89833,123,128,888917
3749576,8399,456,18,247062,124,108,738051


In [231]:
#@title Преобразования dfTrx
%%time
dfTrx['tm'] = dfTrx['tm'].str.split(':').str[0].astype('int')

CPU times: user 14 s, sys: 2.26 s, total: 16.3 s
Wall time: 16.5 s


In [36]:
#@title Определение наиболее используемых mcc
%%time
mcc_by_cid = dfTrx[['cid', 'mcc']].drop_duplicates().astype({'mcc': str}).groupby('cid').agg({'mcc': ' '.join}).sort_values('cid')

large_mcc_count = dfTrx[['cid', 'mcc']].drop_duplicates().groupby('mcc').agg({'cid': 'size'}).sort_values(by = 'cid')
large_mcc_sum = dfTrx[['amt', 'mcc']].groupby('mcc').agg('sum').sort_values(by = 'amt')

most_popular_mcc = set(large_mcc_count[large_mcc_count.cid > 2000].index.to_list()) | set(large_mcc_sum[large_mcc_sum.amt < -1000000000].index.to_list()) | set([123, 125, 130])
len(most_popular_mcc)

CPU times: user 678 ms, sys: 221 ms, total: 899 ms
Wall time: 902 ms


31

In [37]:
#@title Определение наиболее используемых ttc
%%time
ttc_by_cid = dfTrx[['cid', 'ttc']].drop_duplicates().astype({'ttc': str}).groupby('cid').agg({'ttc': ' '.join}).sort_values('cid')

large_ttc_count = dfTrx[['cid', 'ttc']].drop_duplicates().groupby('ttc').agg({'cid': 'size'}).sort_values(by = 'cid')
large_ttc_sum = dfTrx[['amt', 'ttc']].groupby('ttc').agg('sum').sort_values(by = 'amt')

most_popular_ttc = set(large_ttc_count[large_ttc_count.cid > 2000].index.to_list()) | set(large_ttc_sum[(large_ttc_sum.amt < -1000000000) | (large_ttc_sum.amt > 1000000000)].index.to_list())
len(most_popular_ttc)

CPU times: user 548 ms, sys: 113 ms, total: 661 ms
Wall time: 665 ms


25

# **Начало создания df по cid**

In [206]:
#@title Настройки

is_PCA = 1

In [207]:
%%time
df = df_.copy(deep=True)

CPU times: user 1.29 ms, sys: 0 ns, total: 1.29 ms
Wall time: 1.31 ms


In [208]:
#@title MCC
%%time
to_df = df_.copy(deep=True)

# pd.DataFrame(PCA(n_components=83, random_state=0).fit_transform(df))

for mcc_index in most_popular_mcc:
    sum_amt_by_cid_and_mcc = dfTrx[dfTrx.mcc == mcc_index][['cid', 'amt']].groupby('cid').agg({'amt': 'sum'}).sort_values(by = 'cid')
    to_df = to_df.join(sum_amt_by_cid_and_mcc.amt, on = 'cid', how='left').rename(columns = {'amt': str(mcc_index) + '_mcc'})

sum_amt_by_other_mcc = dfTrx[~dfTrx.mcc.isin(most_popular_mcc)].groupby('cid').agg({'amt': 'sum'}).sort_values(by = 'cid')
to_df = to_df.join(sum_amt_by_other_mcc.amt, on = 'cid', how='left').rename(columns = {'amt': 'other_mcc'})

if is_PCA == 2:
    to_df = pd.DataFrame(PCA(n_components=(to_df.shape[1]), random_state=0).fit_transform(to_df.fillna(0).astype('int')))
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 1.66 s, sys: 74.5 ms, total: 1.74 s
Wall time: 1.77 s


In [209]:
#@title TTC
%%time
to_df = df_.copy(deep=True)

for ttc_index in most_popular_ttc:
    sum_amt_by_cid_and_ttc = dfTrx[dfTrx.ttc == ttc_index][['cid', 'amt']].groupby('cid').agg({'amt': 'sum'}).sort_values(by = 'cid')
    to_df = to_df.join(sum_amt_by_cid_and_ttc.amt, on = 'cid', how='left').rename(columns = {'amt': str(ttc_index) + '_ttc'})

sum_amt_by_other_ttc = dfTrx[~dfTrx.ttc.isin(most_popular_ttc)].groupby('cid').agg({'amt': 'sum'}).sort_values(by = 'cid')
to_df = to_df.join(sum_amt_by_other_ttc.amt, on = 'cid', how='left').rename(columns = {'amt': 'other_ttc'})

if is_PCA == 2:
    to_df = pd.DataFrame(PCA(n_components=(to_df.shape[1]), random_state=0).fit_transform(to_df.fillna(0).astype('int')))
    to_df.columns += df.shape[1]
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 1.19 s, sys: 22.9 ms, total: 1.21 s
Wall time: 1.23 s


In [210]:
%%time
sum_amt = dfTrx.groupby('cid').agg({'amt': 'sum'}).sort_values(by = 'cid')
df = df.join(sum_amt.amt, on = 'cid', how='left').rename(columns = {'amt': 'sum_amt'})

df = df.fillna(0).astype('int')

CPU times: user 80.8 ms, sys: 33.9 ms, total: 115 ms
Wall time: 118 ms


In [211]:
#@title Time
%%time
to_df = df_.copy(deep=True)

for hour in range(24):
    sum_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg({'amt': 'sum'}).sort_values(by = 'cid')
    to_df = to_df.join(sum_amt_by_cid_and_hour.amt, on = 'cid', how='left').rename(columns = {'amt': str(hour) + '_tm'})

if is_PCA == 2:
    to_df = pd.DataFrame(PCA(n_components=(to_df.shape[1]), random_state=0).fit_transform(to_df.fillna(0).astype('int')))
    to_df.columns += df.shape[1]
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 7.81 s, sys: 28.1 ms, total: 7.84 s
Wall time: 7.85 s


In [212]:
#@title Date
%%time
to_df = df_.copy(deep=True)

for weekday in range(7):
    sum_amt_by_cid_and_weekday = dfTrx[dfTrx.dt % 7 == weekday][['cid', 'amt']].groupby('cid').agg({'amt': 'sum'}).sort_values(by = 'cid')
    to_df = to_df.join(sum_amt_by_cid_and_weekday.amt, on = 'cid', how='left').rename(columns = {'amt': str(weekday) + '_wd'})

if is_PCA == 2:
    to_df = pd.DataFrame(PCA(n_components=(to_df.shape[1]), random_state=0).fit_transform(to_df.fillna(0).astype('int')))
    to_df.columns += df.shape[1]
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 995 ms, sys: 10.7 ms, total: 1.01 s
Wall time: 1.01 s


In [213]:
df

,64_mcc,65_mcc,130_mcc,72_mcc,73_mcc,75_mcc,77_mcc,79_mcc,19_mcc,20_mcc,...,21_tm,22_tm,23_tm,0_wd,1_wd,2_wd,3_wd,4_wd,5_wd,6_wd
cid,,,,,,,,,,,,,,,,,,,,,
0,-98517,-411791,0,-235227,-30178,-292138,-195288,-49338,-42667,-253538,...,NaN,NaN,NaN,-1503018.0,-1944919.0,-1993280.0,-1462478.0,-3922469.0,-1156890.0,-256102.0
1,0,-118491,0,0,0,0,-6717,0,-10102,-202246,...,NaN,NaN,NaN,-418377.0,-980780.0,-762884.0,-1037652.0,-54254.0,-1646770.0,327954.0
2,-363746,-135444,0,0,0,0,0,0,0,-185907,...,NaN,NaN,NaN,-2147341.0,-201523.0,-213613.0,-916211.0,-63631.0,-325482.0,-2045809.0
3,0,0,0,0,0,0,-69392,0,0,0,...,NaN,NaN,NaN,-2878764.0,-1461509.0,-1478492.0,-4990396.0,-3881429.0,-6147942.0,-5740670.0
4,0,0,0,0,0,0,0,0,0,-29186,...,NaN,NaN,NaN,-626389.0,-317599.0,-186076.0,-591087.0,-548931.0,-447977.0,-515219.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8395,-72305,-169165,0,-13445,-149525,-202835,0,0,-6731,-184398,...,NaN,NaN,NaN,982368.0,-434873.0,-166370.0,-2692095.0,-1121356.0,497071.0,442757.0
8396,0,0,0,0,0,0,0,0,0,-1126,...,NaN,NaN,NaN,NaN,NaN,NaN,-10113.0,-34802.0,-13475.0,NaN
8397,0,0,0,0,0,0,-83081,-67357,0,-137129,...,NaN,NaN,NaN,395621.0,150887.0,86126.0,-341616.0,-16581.0,710021.0,-1597920.0


In [214]:
#@title PCA
%%time

df = df.fillna(0).astype('int')
if is_PCA in [1, 2]:
    df.columns = [str(c) for c in df.columns]
    df = pd.DataFrame(PCA(n_components=df.shape[1] // 1, random_state=0).fit_transform(df))

CPU times: user 125 ms, sys: 13.2 ms, total: 138 ms
Wall time: 91.3 ms


In [215]:
df

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,-9.635892e+06,1.810003e+06,-8.274092e+05,-3.864889e+06,1.821011e+06,3.780244e+05,954549.347792,1.927393e+06,-5.452917e+05,1.106880e+06,...,4.721187e-09,5.284337e-10,7.335986e-10,-3.163837e-09,1.184125e-09,1.841140e-09,8.326542e-10,-7.139028e-10,-1.376936e-09,3.383393e-10
1,-1.220273e+07,-6.488264e+06,2.008829e+06,-3.284493e+06,1.518890e+06,-1.051294e+06,-713498.211645,-1.322394e+06,2.989629e+05,-1.086499e+06,...,1.481288e-09,2.038644e-10,4.108560e-09,-1.224877e-09,1.163945e-09,8.865356e-10,-4.282383e-09,4.614588e-09,-1.049118e-09,1.842523e-09
2,-9.045250e+06,-5.344161e+06,1.772209e+06,-4.636468e+06,2.547606e+06,-1.180131e+05,-469079.570969,-1.931100e+05,-1.242820e+06,-1.680085e+05,...,-2.477230e-09,7.635288e-10,-3.214775e-09,1.656979e-10,-4.056198e-10,-2.490814e-09,-1.111115e-09,5.423641e-10,2.342174e-09,-7.197570e-10
3,-8.511675e+06,3.115063e+07,-3.037268e+06,-1.299993e+07,-4.035562e+06,-7.751334e+05,-273069.972428,-2.604241e+06,-4.246019e+05,-5.551770e+05,...,-1.710693e-08,-6.905068e-09,6.240420e-09,1.184406e-08,2.101213e-09,-2.675852e-09,1.515846e-09,1.231176e-09,3.263325e-10,2.900596e-10
4,-1.365748e+07,-1.014189e+07,2.080796e+06,-4.936185e+06,3.217227e+06,-1.456547e+05,-692403.419785,-6.938215e+05,-2.275623e+05,-3.633321e+05,...,-6.390750e-09,1.472521e-08,-5.924698e-09,-2.063359e-09,-1.318668e-08,1.368511e-09,3.185874e-09,2.906280e-09,4.509991e-09,-1.332875e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8395,-2.821788e+06,-7.443950e+06,4.894812e+06,-2.744132e+06,3.538193e+05,-1.302622e+06,-305059.262128,-7.148561e+05,-1.357273e+06,5.921810e+05,...,9.184725e-12,-5.435286e-10,-8.081582e-11,2.773873e-10,3.711741e-10,6.165040e-11,2.230256e-10,2.610777e-12,-2.870661e-11,2.717889e-11
8396,-1.423341e+07,-1.330341e+07,3.889465e+06,-4.599431e+06,3.177716e+06,-4.966506e+05,-966719.127198,-1.505157e+06,-8.252990e+04,-7.638826e+05,...,-1.845916e-10,-1.366158e-11,-1.455858e-10,5.650481e-11,4.120206e-11,2.919489e-11,2.304428e-11,1.128202e-10,1.084863e-10,1.967294e-11
8397,-1.213381e+07,-1.040399e+07,4.641998e+06,-3.131581e+06,-4.203660e+05,-1.273708e+06,-607392.708277,-1.186500e+06,-1.845441e+05,-4.773558e+05,...,-2.417293e-10,1.566930e-10,2.595416e-11,1.021997e-10,-1.182402e-10,-5.848912e-11,-5.068609e-11,5.636920e-11,9.685668e-11,1.600866e-11
8398,-1.336396e+07,-7.507054e+06,2.802397e+06,-5.567010e+06,2.336428e+06,-3.246033e+05,-773296.344460,-1.213810e+06,2.184552e+04,-7.429663e+05,...,-1.151442e-10,-3.877625e-11,-6.596241e-11,9.984642e-12,8.796572e-11,1.243729e-12,1.687385e-11,4.086149e-11,4.076702e-11,-1.686507e-11


# **Обучение**

In [217]:
%%time
df.columns = [str(c) for c in df.columns]

trainX, testX = df.join(trainY.drop('gender', axis=1), how='inner'), df.join(testY.drop('gender', axis=1), how='inner') # split observations into train/test
trainX0, testX0, trainY0, testY0 = train_test_split(trainX, trainY, test_size=0.3, random_state=0) # split train observation into local train/test sets (to score performance)

model = LR(random_state=0, n_jobs=-1, max_iter=1000)                             # always seed your experiments for reproducibility
# model = RFC()

print('Local AUC ROC', roc_auc_score(testY0.gender, model.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]))       # fit the model and show test error

Local AUC ROC 0.7671152998236332
CPU times: user 103 ms, sys: 114 ms, total: 216 ms
Wall time: 4.35 s


In [189]:
tmr.ShowTime()

Runtime is 3107 sec > 120 sec limit!!!


In [220]:
model = LR(random_state=0, n_jobs=-1, max_iter=1000).fit(trainX, trainY.gender)  # now train again, on the full training set
pY = pd.DataFrame(model.predict_proba(testX)[:,1], columns=['gender']) > 0.52
pY.index.name = 'cid'
# pY.reset_index().to_csv('💳Baseline.csv', index=False, float_format='%d')

In [221]:
pY.sum()

gender    510
dtype: int64

In [222]:
pY.reset_index().to_csv('💳Baseline.csv', index=False, float_format='%d')